<a href="https://colab.research.google.com/github/dhirajsuvarna/clustering/blob/master/notebook/Clustering_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clustering on 3D Dataset 

This notebook is used to run the python scripts for 
1. Generate **latent vectors** using autoencoders 
1. Perform clustering



In [8]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
cd /content/drive/My Drive/Projects/clustering/clustering_3d_data

/content/drive/My Drive/Projects/clustering/clustering_3d_data


In [21]:
#! git clone https://github.com/dhirajsuvarna/clustering.git
#! git config --global user.email "dhiraj.suvarna@gmail.com"
#! git stash
! git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/dhirajsuvarna/clustering
   1c2eb77..8240578  master     -> origin/master
Updating 1c2eb77..8240578
Fast-forward
 clustering_3d_data/train.py | 1 +
 1 file changed, 1 insertion(+)


## Install Python Packages

In [11]:
!pip install open3d==0.9.0

In [12]:
! pip install ninja

In [13]:
! pip uninstall tensorflow -y

In [14]:
! pip install tensorboard 

## Create Dataset Split

In [4]:
cd /content/drive/My Drive/Projects/pointnet/pointnet.pytorch/dataset_utils/pcd_utils

/content/drive/My Drive/Projects/pointnet/pointnet.pytorch/dataset_utils/pcd_utils


In [ ]:
! python create_train_test_shuffle.py --input /content/drive/My\ Drive/Projects/dataset/shapenet_pcd --split_percent 99 --ext .pcd

Arguments:Namespace(ext='.pcd', input='/content/drive/My Drive/Projects/dataset/shapenet_pcd', labeled_split=False, split_percent=99)
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/airplane
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/bag
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/cap
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/car
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/chair
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/earphone
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/guitar
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/knife
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/lamp
Creating split of : /content/drive/My Drive/Projects/dataset/shapenet_pcd/laptop
Creating split of : /content/drive/My Drive/Projects/dataset/shap

## Training 

In [22]:
cd /content/drive/My\ Drive/Projects/clustering/clustering_3d_data


/content/drive/My Drive/Projects/clustering/clustering_3d_data


In [24]:
! cat train.py

import torch 
import argparse
import os
import random
from torch.utils.data import DataLoader
from torch import optim

from dataprep.dataset import PointCloudDataset
from model.model import PCAutoEncoder
from model.model_fxia22 import PointNetAE
from torch.utils.tensorboard import SummaryWriter
import numpy as np
from util import pointutil
import clustering
import runmanager
import json

if torch.cuda.is_available():
    from chamfer_distance.chamfer_distance_gpu import ChamferDistance # https://github.com/chrdiller/pyTorchChamferDistance
else:
    from chamfer_distance.chamfer_distance_cpu import ChamferDistance # https://github.com/chrdiller/pyTorchChamferDistance

#########################################################################
# SHOULD BE STRICTLY REFACTORED - this is not acceptable here
def trimfilenames(iFileName):
    # "F:\projects\ai\pointnet\dataset\DMUNet_OBJ_format\dataset_PCD_5000\Switch\Switch_4.pcd" -> "Switch\Switch_4.pcd"
    pathComps = os.path.normpath(iFile

In [26]:
! python train.py --batch_size 64 \
                  --num_points 2048 \
                  --num_workers 4 \
                  --nepoch 150 \
                  --model_type fxia \
                  --dataset_path /content/drive/My\ Drive/Projects/dataset/shapenet_pcd \
                  --latent_vector "" \
                  --filenames "" 2>&1 | tee train_fxia_shapenet.log \


Streaming output truncated to the last 5000 lines.
Epoch: 132, Iteration: 249, Loss: 0.0017618241254240274
Epoch: 132, Iteration: 250, Loss: 0.0018679737113416195
Epoch: 132, Iteration: 251, Loss: 0.001681943191215396
Epoch: 132, Iteration: 252, Loss: 0.001750099123455584
Epoch: 132, Iteration: 253, Loss: 0.0019669514149427414
Epoch: 132, Iteration: 254, Loss: 0.0018086706986650825
Epoch: 132, Iteration: 255, Loss: 0.0017529085744172335
Epoch: 132, Iteration: 256, Loss: 0.0020318590104579926
Epoch: 132, Iteration: 257, Loss: 0.0018791410839185119
Epoch: 132, Iteration: 258, Loss: 0.0018744241679087281
Epoch: 132, Iteration: 259, Loss: 0.001916710170917213
Epoch: 132, Iteration: 260, Loss: 0.0018355955835431814
Epoch: 132, Iteration: 261, Loss: 0.0019541974179446697
Epoch: 132, Iteration: 262, Loss: 0.0018978482112288475
Epoch: 132, Iteration: 263, Loss: 0.0018578553572297096
Epoch: 132, Iteration: 264, Loss: 0.0017857670318335295
Epoch: 132, Iteration: 265, Loss: 0.0017355906311422586


Perform Clustering using Mean Shift Algorithm


In [ ]:
! cat train.py

In [44]:
! python train.py --batch_size 0 \
                  --num_points 0 \
                  --num_workers 0 \
                  --nepoch 0 \
                  --model_type fxia \
                  --dataset_path /content/drive/My\ Drive/Projects/dataset/shapenet_pcd \
                  --only_clustering \
                  --latent_vector "/content/drive/My Drive/Projects/clustering/clustering_3d_data/saved_models_bkp/best_latent_vector_142.pth" \
                  --filenames "/content/drive/My Drive/Projects/clustering/clustering_3d_data/saved_models_bkp/best_filenames_142.pth" 


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
Input Arguments : Namespace(batch_size=0, dataset_path='/content/drive/My Drive/Projects/dataset/shapenet_pcd', filenames='/content/drive/My Drive/Projects/clustering/clustering_3d_data/saved_models_bkp/best_filenames_142.pth', latent_vector='/content/drive/My Drive/Projects/clustering/clustering_3d_data/saved_models_bkp/best_latent_vector_142.pth', load_saved_model='', model_type='fxia', nepoch=0, num_points=0, num_workers=0, only_clustering=True, start_epoch_from=0)
Random Seed:  7133
Performing Clustering...
Clusters file generated
Writing to Tensor

In [13]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=f5e49bb460be304f5bf136207c526c7f83b4f4a92853ca0f8f37de6732480306
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.6 GB  | Proc size: 160.4 MB
GPU RAM Free: 16280MB | Used: 0MB | Util   0% | Total 16280MB


In [ ]:
#if utilization is greater than 0% try to kill your machine, using the code below.
!kill -9 -1

In [ ]:
! ls -l  | tee /content/sample.out

total 24
drwx------ 3 root root 4096 May 22 10:03 chamfer_distance
drwx------ 3 root root 4096 May 22 10:03 dataprep
-rw------- 1 root root 3603 May 22 09:58 infer.py
drwx------ 3 root root 4096 May 22 10:03 model
drwx------ 2 root root 4096 May 22 10:37 saved_models
-rw------- 1 root root 3972 May 22 10:33 train.py
